#### READ ME
##### Чтение геометрии: GeoJSON - ID, gauge_name, геометрия
##### Чтение гидрологии: CSV - имя файла (ID поста), Дата, Расход
##### По геометрии в WGS 84 считается площадь для расчёта слоя стока

In [1]:
import pandas as pd
import fiona # требуется LINUX
import geopandas as gpd
import glob
import numpy as np

#### Указание обязательных путей хранения файлов

In [2]:
hydrology_data_path = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/Uni_Input/hydrological_series/' # путь до папки с файлами расходов
geometry_data_path = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/Uni_Input/geometry/' # путь до файла GeoJSON. В файлах 
path_to_results = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/code/camels_ru/results/'

#!TODO заменить на PostgresSQL
HydroATLAS_data_path = '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/HydroAtlas/Data/' # путь до .gdb файла HydroATLAS

In [3]:
def open_files_for_CAMELS(file_path_hydrology, file_path_geometry):# geometry = False, hydrology = False):
    
    """
    function which reads file with given format:
    .geojson for geometry
    
    Return GeoDataFrame with 4 columns
    
    'gauge_name', 'ID', 'geometry' and 'area'
    
    Input file should have same column names !
    
    .csv for hydrology data
    
    In this case it's return list of dataframes: each dataframe for gauge
    DataFrame consist 3 columns
    
    'date', 'layer', 'discharge', 'baseflow'
    """
           
    def polygon_area(lats, lons, radius = 6378137):
        """
        Computes area of spherical polygon, assuming spherical Earth. 
        Returns result in ratio of the sphere's area if the radius is specified.
        Otherwise, in the units of provided radius.
        lats and lons are in degrees.
        """
        from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
        lats = np.deg2rad(lats)
        lons = np.deg2rad(lons)

        # Line integral based on Green's Theorem, assumes spherical Earth

        #close polygon
        if lats[0]!=lats[-1]:
            lats = append(lats, lats[0])
            lons = append(lons, lons[0])

        #colatitudes relative to (0,0)
        a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
        colat = 2*arctan2( sqrt(a), sqrt(1-a) )

        #azimuths relative to (0,0)
        az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

        # Calculate diffs
        # daz = diff(az) % (2*pi)
        daz = diff(az)
        daz = (daz + pi) % (2 * pi) - pi

        deltas=diff(colat)/2
        colat=colat[0:-1]+deltas

        # Perform integral
        integrands = (1-cos(colat)) * daz

        # Integrate 
        area = abs(sum(integrands))/(4*pi)

        area = min(area,1-area)
        if radius is not None: #return in units of radius
            return area * 4 * pi* radius**2 / 10**6
        else: #return in ratio of sphere total area
            return area / 10**6

    from shapely.geometry import Polygon, MultiPolygon
        
    selected_files = glob.glob(file_path_geometry + '*.geojson') # Select every file from folder. There's should be only one
# написать потом обработку отдельных GeoJSON для каждого водосбора        
    file = open(selected_files[0])
    df = gpd.read_file(file)
    file.close()

    for geometry_row in range(len(df.geometry)): 
        if type(df.geometry[geometry_row]) == MultiPolygon: # It's simplify next manipulations with geometry Data

            from shapely.ops import cascaded_union

            df.geometry[geometry_row] = cascaded_union(df.geometry[geometry_row])

        elif type(df.geometry[geometry_row]) == Polygon:
            pass

        else:
            print('There is not supported type of geometry: {}. Should be shapely.Polygon or shapely.MultiPolygon'.format(type(geometry_row)))

    df['Area'] = [polygon_area(lats = df.geometry[row].exterior.coords.xy[1], lons = df.geometry[row].exterior.coords.xy[0]) for row in range(len(df.geometry))]
    
    
################################################################################################################################################################


    selected_files = glob.glob(file_path_hydrology + '*.csv') # For each gauge there's own file with ID of watershed
    file = [pd.read_csv(file) for file in selected_files]
    ID = [int(ID[-9:-4]) if int(ID[-9:-4]) != 99999 else np.NaN for ID in selected_files] # select only identified ID's

    for gauge_dataframe in file:
        gauge_dataframe['date'] = pd.to_datetime(gauge_dataframe['date']) # assign datetime format for date column
    
    
    for i, gauge_dataframe in enumerate(file):
        for j in range(len(df)):
            fnd = True
            while fnd:

                fnd = False

                if df.ID[j] == ID[i]:
                    layer = 86400 * gauge_dataframe.discharge * 10**9 / (df.Area[j] * 10**12)
                    gauge_dataframe.insert(loc = 1, column = 'layer', value = layer) 

                    fnd = True
                    break

    for gauge_dataframe in file:
        gauge_dataframe['baseflow'] = np.NaN # add column for baseflow

    file = [gauge for i, gauge in enumerate(file) if np.isnan(ID[i]) == False] # return only files with given ID's
    ID = [i for i in ID if np.isnan(i) == False]

    return file, df, ID # returning list of DataFrames and List of Common ID's


In [4]:
hydrology_data, geometry_data, ID = open_files_for_CAMELS(hydrology_data_path, geometry_data_path)
# geometry_data = open_files_for_CAMELS(, geometry = True)

In [5]:
hydrology_data[0]

,date,layer,discharge,baseflow
0,1970-01-01,0.116424,0.97,NaN
1,1970-01-02,0.114023,0.95,NaN
2,1970-01-03,0.114023,0.95,NaN
3,1970-01-04,0.112823,0.94,NaN
4,1970-01-05,0.106822,0.89,NaN
...,...,...,...,...
17162,2016-12-27,1.440296,12.00,NaN
17163,2016-12-28,1.368282,11.40,NaN
17164,2016-12-29,1.392287,11.60,NaN
17165,2016-12-30,1.308269,10.90,NaN


In [6]:
def hydro_signatures_for_CAMELS(hydro_data, path_to_results , ID): # ID IS TEMPORARY
    import math
    import time
    list_of_gauges = hydro_data
    ###########################################################################################################################################
    
    def split_by_year(list_of_something, number_of_NaN):
    
        """
        Данная функция разбивает лист датафреймов по годам
        и выкидывает года, где NaN регулируется переменной number_of_NaN

        Колонка Date в датафрейме внутри листа - даты
        и она обязательна. 
        Столбцы могут называться в соответствии с пожеланиями

        """
        splitted_list = list()

        for i, gauge in enumerate(list_of_something):
            """
            From list of lists we are calling another list 
            that consist list of dataframes

            """
            year_split = list()
            unique_years = gauge.date.dt.year.unique()

            for year in unique_years:
                year_split.append(gauge[gauge.date.dt.year == year].reset_index(drop = True))

            for i in range(len(year_split)-1, -1, -1):
                if sum(year_split[i].discharge.isna()) > number_of_NaN:
                    del year_split[i]
                else:
                    pass

            splitted_list.append(year_split)

        return splitted_list

    ###########################################################################################################################################
    
    """
    Q mean for observations on every valid gauge. Dimension - mm/day

    """

    def mean_for_gauge(list_of_gauges):
        gauges_Q_mean = [np.nanmean(i.layer) for i in list_of_gauges]
        print('Mean layer of discharge calculation')
        return gauges_Q_mean
    
    ###########################################################################################################################################
    
    def slope_fdc_gauge(list_of_gauges):
        slope_fdc = []
        for i, gauge in enumerate(list_of_gauges):
            if (np.nanpercentile(gauge.layer.to_numpy(), q = 100 - 33) != 0) & (np.nanpercentile(gauge.layer.to_numpy(), q = 100 - 66) != 0):
                slope_fdc.append((math.log(np.nanpercentile(gauge.layer.to_numpy(), q = 100 - 33)) - math.log(np.nanpercentile(gauge.layer.to_numpy(), q = 100 - 66)))/(0.66-0.33))
            else:
                slope_fdc.append(np.NaN)
        print('Flow duration curve calculation')
        return slope_fdc
    
    ###########################################################################################################################################
    
    
    import numba

    """
    Расчёт ведётся для листа в котором нет пропусков. 
    В случае их наличия в общем ряду, он разбивается
    на n-рядов в зависимости от разбиений

    Расчёт BFI далее будет производиться для отдельно 
    взятого года наблюдений

    """
    ###################################################################

    """
    First pass

    """
    @numba.jit(nopython = True)
    def FirstPass(Q, alpha):

        q_f_1 = [np.float64(np.NaN) for i in Q]
        q_b_1 = [np.float64(np.NaN) for i in Q]

        q_f_1[0] = Q[0]

        for j in range(len(Q)-1):
            """
            for every split calculate quick flow

            """
            q_f_1[j+1] = alpha * q_f_1[j] + 0.5 * (1 + alpha) * (Q[j+1] - Q[j])

        for j in range(len(Q)):
            if q_f_1[j] < 0:
                q_b_1[j] = Q[j]
            else:
                q_b_1[j] = Q[j] - q_f_1[j]

        Q_forward_1 = [q_f_1, q_b_1]

        return Q_forward_1

    ###################################################################
    """
    Backward pass

    """
    @numba.jit(nopython = True)
    def BackwardPass(Q_forward_1, alpha):

        """
        Здесь Q - n-мерный лист в зависимости от числа разбиений
        """

        Qq = Q_forward_1[0]
        Qb = Q_forward_1[1]

        q_f_2 = [np.float64(np.NaN) for i in Qq]
        q_b_2 = [np.float64(np.NaN) for i in Qb]


        "last value of forward step - first in backward step"

        q_f_2[-1] = Qb[-1]

        for j in range(len(Qq)-2, -1, -1):
            q_f_2[j] = alpha * q_f_2[j+1] + 0.5 * (1 + alpha) * (Qb[j] - Qb[j+1])

        for j in range(len(Qq)-1, -1, -1):
            if q_f_2[j] < 0:
                q_b_2[j] = Qb[j]
            else:
                q_b_2[j] = Qb[j] - q_f_2[j]

        Q_backward = [q_f_2, q_b_2]

        return Q_backward

    ###################################################################
    """
    Forward pass

    """
    @numba.jit(nopython = True)
    def ForwardPass(Q_backward, alpha):

        Qq = Q_backward[0]
        Qb = Q_backward[1]

        q_f_3 = [np.float64(np.NaN) for i in Qq]
        q_b_3 = [np.float64(np.NaN) for i in Qb]


        "Теперь первая величина предыдущего шага - первая и здесь"

        q_f_3[0] = Qb[0]

        for j in range(len(Qb)-1):

            q_f_3[j+1] = alpha * q_f_3[j] + 0.5 * (1 + alpha) * (Qb[j+1] - Qb[j])

        for j in range(len(Qb)):
            if q_f_3[j] < 0:
                q_b_3[j] = Qb[j]
            else:
                q_b_3[j] = Qb[j] - q_f_3[j]

        Q_forward = [q_f_3, q_b_3]

        return Q_forward

    ###################################################################
    """
    BFI calculations for given alpha

    """
    @numba.jit(nopython = True)
    def BFI_calc(Q, alpha, passes, reflect):
        """
        we reflect the first reflect values and the last reflect values.  
        this is to get rid of 'warm up' problems © Anthony Ladson
        """ 
        Qin = Q

        "reflect our lists"

        if len(Q)-1 > reflect:
            Q_reflect = np.array([np.float64(np.NaN) for _ in range(len(Q) + 2 * reflect)], dtype = np.float64)

            Q_reflect[:reflect] = Q[(reflect):0:-1]
            Q_reflect[(reflect):(reflect + len(Q))] = Q
            Q_reflect[(reflect + len(Q)):(len(Q) + 2 + 2 * reflect)] = Q[len(Q)-2:len(Q) - reflect - 2:-1]

        else:        
            Q_reflect = np.array([np.float64(np.NaN) for _ in range(len(Q))], dtype = np.float64)                 
            Q_reflect = Q

        Q1 = FirstPass(Q_reflect, alpha)

        "how many backwards/forward passes to we need © Anthony Ladson"

        n_pass = round(0.5 * (passes -1))

        BackwardPass(Q1, alpha)

        for i in range(n_pass):
            Q1 = ForwardPass(BackwardPass(Q1, alpha), alpha)

        ################# end of passes  ##############################
        if len(Q)-1 > reflect:
            Qbase = Q1[1][reflect:(len(Q1[1])-reflect)]
            Qbase = [0 if j < 0 else j for j in Qbase]
        else:
            Qbase = Q1[1]
            Qbase = [0 if j < 0 else j for j in Qbase]

        bfi = 0
        mean_for_period = 0

        if np.mean(Qin) == 0:
            bfi = 0
        else:
            for j in Qbase:
                mean_for_period += j/np.mean(Qin)
            bfi = mean_for_period/len(Qbase)

        return bfi, Qbase

    """
    BFI calculations for 1000 alpha between 0.9 and 0.98

    """

    import random
    @numba.jit(nopython = True)
    def BFI_calc_1000(Q, passes, reflect):
        """
        Расчёт проводится для 1000 случайных значений alpha
        в диапазоне он 0.9 до 0.98

        we reflect the first reflect values and the last reflect values.  
        this is to get rid of 'warm up' problems © Anthony Ladson
        """ 

        random.seed(1996)
        alpha_coefficients = [np.float64(random.uniform(0.9, 0.98)) for i in range(1000)]

        Q = np.array([np.float64(i) for i in Q], dtype = np.float64)
        Qin = Q

        "reflect our lists"

        if len(Q)-1 > reflect:
            Q_reflect = np.array([np.float64(np.NaN) for _ in range(len(Q) + 2 * reflect)], dtype = np.float64)

            Q_reflect[:reflect] = Q[(reflect):0:-1]
            Q_reflect[(reflect):(reflect + len(Q))] = Q
            Q_reflect[(reflect + len(Q)):(len(Q) + 2 + 2 * reflect)] = Q[len(Q)-2:len(Q) - reflect - 2:-1]

        else:        
            Q_reflect = np.array([np.float64(np.NaN) for _ in range(len(Q))], dtype = np.float64)                 
            Q_reflect = Q

        bfi_record = []
        Qbase_record = []

        for i, alpha in enumerate(alpha_coefficients):

            Q1 = FirstPass(Q_reflect, alpha)

            "how many backwards/forward passes to we need © Anthony Ladson"

            n_pass = round(0.5 * (passes -1))

            BackwardPass(Q1, alpha)

            for i in range(n_pass):
                Q1 = ForwardPass(BackwardPass(Q1, alpha), alpha)


            ################# end of passes  ##############################
            if len(Q)-1 > reflect:
                Qbase = Q1[1][reflect:(len(Q1[1])-reflect)]
                Qbase = [0 if j < 0 else j for j in Qbase]
            else:
                Qbase = Q1[1]
                Qbase = [0 if j < 0 else j for j in Qbase]

            Qbase_record.append(np.array(Qbase, dtype=np.float64))

            bfi = 0
            mean_for_period = 0

            if np.mean(Qin) == 0:
                bfi = 0
            else:
                for j in Qbase:
                    mean_for_period += j/np.mean(Qin)
                bfi = mean_for_period/len(Qbase)

            bfi_record.append(np.float64(bfi))

        """
        After 1000 calculations function return
        mean value out of 1000

        And "mean" hygrograph of baseflow

        """

        # mean BFI out of 1000

        bfi_mean = 0
        for i in bfi_record:
            bfi_mean += i
        bfi_mean = bfi_mean/len(bfi_record)

        # mean hydrograph out of 1000 calculations

        Qbase_mean = [np.float64(0) for i in range(len(Qbase))]

        for Qbase_temp in Qbase_record:
            for i, value in enumerate(Qbase_temp):
                Qbase_mean[i] += value

        Qbase_mean = [np.float64(i/len(Qbase_record)) for i in Qbase_mean]
        
        return bfi_mean, Qbase_mean
    
    
    ###########################################################################################################################################
    
    def clump_array(a):

        """
        Разбить период наблюдений на куски, в которых нет NaN
        """

        return [np.float64(a[s]) for s in np.ma.clump_unmasked(np.ma.masked_invalid(a))]
    
    ###########################################################################################################################################
    
    
    def BFI_calculation_for_lists(Valid_gauges_Q_cms, Number_of_NaN):
        # 
        every_gauge_split_by_year = split_by_year(Valid_gauges_Q_cms, Number_of_NaN)
        # разбитие столбца из датафрейма на промежутки без NaN
        every_gauge_split_by_year_np = [[clump_array(year.discharge.to_numpy()) for year in gauge] for gauge in every_gauge_split_by_year]

        clump_bfi = [[[[] for clump in year]  for year in gauge] for gauge in every_gauge_split_by_year_np]
        weights_of_clump = [[[[] for clump in year]  for year in gauge] for gauge in every_gauge_split_by_year_np]
        clump_Qbase = [[[[] for clump in year]  for year in gauge] for gauge in every_gauge_split_by_year_np]
        
        print('Base flow index calculation')
        for i, gauge in enumerate(every_gauge_split_by_year_np):
            start = time.time()
            for j, year in enumerate(gauge):
                for k, clump in enumerate(year):
                    # проверка, что кусок между NaN больше, длины сравнения
                    if type(clump) == np.ndarray:
                        clump_bfi[i][j][k] = BFI_calc_1000(clump, 3, 30)[0]
                        weights_of_clump[i][j][k] = len(clump)/len(every_gauge_split_by_year[i][j].discharge)
                        clump_Qbase[i][j][k] = BFI_calc_1000(clump, 3, 30)[1]
                    else:
                        clump_bfi[i][j][k] = np.NaN
                        weights_of_clump[i][j][k] = np.NaN
                        clump_Qbase[i][j][k] = [np.NaN]
            stop = time.time()
            print('Расчёт на {} посту закончилась за {} секунд'.format(i, round(stop - start)))
        weighted_bfi = [np.mean([np.sum([bfi * weights_of_clump[i][j][k] for k, bfi in enumerate(year)]) for j, year in enumerate(gauge)]) if len(gauge) > 1 else np.NaN for i, gauge in enumerate(clump_bfi)]


        """
           Заполнение столбца baseflow восстанавливая разбитые на куски без NaN наблюдения за стоком
        """

        from itertools import groupby
        def bfi_recover(year_of_observation, bfi_values):

            """
            Восстановления ряда baseflow с учётом ранее "выбитых" пропусков
            """
            Qbase_full = [list(group) for key, group in groupby(year_of_observation.discharge.to_numpy(), key=np.isnan)] # разбитие на группы по признаку. Получается n листов. NaN и !NaN разбиты по листам
            mask_to_refill = [~np.isnan(i).any() for i in Qbase_full] # индексируем их как одинокие булевые величины

            position_index = -1
            for j, mask in enumerate(mask_to_refill):
                """
                Так как первичный разрезанный лист из функции clumped_array имеет в себе количество листов, такое же как и True, то восстанавливаем в соответствии

                Аргумент position_index подбирается в соответствии с присутсвием с True и адресуется к нужному куску в Qbase

                Всё перезаписывается в лист, где в перезаписываемых значениях эквивалентные по длине значения стока
                """
                if len(Qbase_full) > 1:
                    if mask:
                        position_index += 1
                        Qbase_full[j] = bfi_values[position_index]
                    else:
                        position_index = position_index
                        Qbase_full[j] = Qbase_full[j]
                else:
                    Qbase_full[j] = [item for sublist in bfi_values for item in sublist]

            if len(Qbase_full) > 1:
                Qbase_full = [item for sublist in Qbase_full for item in sublist] # лист листов в лист с учётом NaN
            else:
                Qbase_full = [item for sublist in Qbase_full for item in sublist]

            return Qbase_full

        for i, gauge in enumerate(every_gauge_split_by_year_np):
            for j, year in enumerate(gauge):
                for k, clump in enumerate(year):
                    every_gauge_split_by_year[i][j].loc[:, 'baseflow'] = bfi_recover(every_gauge_split_by_year[i][j], clump_Qbase[i][j])
        print('Base Flow Index splitting')
        return weighted_bfi, every_gauge_split_by_year
    
    
    ###########################################################################################################################################
    
    
    def hfd_mean_for_gauges(Valid_gauges_cms, number_of_NaN):
        
        hydro_data = Valid_gauges_cms
        import datetime

        every_gauge_split_by_year_hfd = split_by_year(hydro_data, number_of_NaN)

        hfd = [[[] for year in range(len(gauge)-1)] for gauge in every_gauge_split_by_year_hfd]

        for i, gauge in enumerate(every_gauge_split_by_year_hfd):
            for j in range(len(gauge)-1):

                if (datetime.datetime(gauge[0].date[0].year, 10, 1) >= gauge[0].date[0]) & \
                (len(gauge[0]) > 91): # check if first split is starting at 1 of october or earlier and have no spaces between 1.10.i and 31.12.i

                    # if it's suffice criteria then we'll split it by hydrological years: two consecutive years from 1.10.i tp 1.10.i+1
                    hfd[i][j] = pd.concat([gauge[j][gauge[j].date >= datetime.datetime(gauge[j].date[0].year, 10, 1)], 
                       gauge[j+1][gauge[j+1].date < datetime.datetime(gauge[j+1].date[0].year, 10, 1)]]).reset_index(drop = True)
                else:
                    hfd[i][j].append(np.NaN)

        hfd_value = [[] for _ in every_gauge_split_by_year_hfd]

        for i, gauge in enumerate(hfd):
            for j, hydrological_year in enumerate(gauge):
                half_discharge = 0
                half_sum = gauge[j].discharge.dropna().sum()/2

                for k, discharge in enumerate(hydrological_year.discharge):
                    half_discharge += discharge
                    if half_discharge > half_sum:
                        hfd_value[i].append(k + 1)
                        break
        hfd_value = [np.nanmean(gauge_dates) for gauge_dates in hfd_value]

        print('Half flow date calculation')

        return hfd_value
        
    
    ###########################################################################################################################################
    
    
    def Q5_for_gauge(list_of_gauges):
        Q5 = [np.nanpercentile(gauge.layer.to_numpy(), q = 5) for gauge in list_of_gauges]
        print('5% quantile calculation')
        return Q5
    
    
    ###########################################################################################################################################
    
    
    def Q95_for_gauge(list_of_gauges):
        Q95 = [np.nanpercentile(gauge.layer.to_numpy(), q = 95) for gauge in list_of_gauges]
        print('95% quantile calculation')
        return Q95
    
    
    ###########################################################################################################################################
    
    
    def high_q_frequency(Valid_gauges_in_mm): #Anthony Ladson version
    
        hydro_data = Valid_gauges_in_mm

        high_q_f = list()

        for gauge in hydro_data:
            if len(gauge) > 1:
                high_q_f.append(
                len(gauge.layer.dropna()[gauge.layer.dropna() > gauge.layer.dropna().median() * 9])/len(gauge.layer.dropna()) * 365.25
                )
            else:
                high_q_f.append(np.NaN)

        print('High discharge frequency calculation')

        return high_q_f    
    
    ###########################################################################################################################################
    
    
    def high_q_duration(Valid_gauges_in_mm, number_of_Nan):    
        hydro_data = Valid_gauges_in_mm

        every_gauge_split_by_year_mm = split_by_year(hydro_data, number_of_Nan)

        FREQ_INSTANCE = list()
        SEQ_OF_FREQ = list()
        LEN_OF_SEQ = [[[] for year in gauge] for gauge in every_gauge_split_by_year_mm]


        for i, gauge in enumerate(every_gauge_split_by_year_mm):
            median_x9 = hydro_data[i].layer.dropna().median() * 9
            FREQ_INSTANCE.append([every_gauge_split_by_year_mm[i][j][every_gauge_split_by_year_mm[i][j].layer > median_x9] 
                              for j in range(len(every_gauge_split_by_year_mm[i]))])

            SEQ_OF_FREQ.append([[d for _, d in FREQ_INSTANCE[i][j].groupby(FREQ_INSTANCE[i][j].index - np.arange(len(FREQ_INSTANCE[i][j])))]
                                if FREQ_INSTANCE[i][j].empty != True else [0]
                           for j in range(len(FREQ_INSTANCE[i]))])
            for k, seq in enumerate(SEQ_OF_FREQ[i]):
                if len(seq) == 0:
                    LEN_OF_SEQ[i][k] = [np.NaN]
                else:
                    LEN_OF_SEQ[i][k] = [len(seq[ii])
                                        if type(seq[ii]) != int
                                        else 0
                                        for ii in range(len(seq))]
        high_q_d = list()

        for i, gauge in enumerate(LEN_OF_SEQ):
            if len(gauge) > 1:
                if all([True if i == 0 else False for i in [item for sublist in LEN_OF_SEQ[i] for item in sublist] ]):
                    high_q_d.append(0)
                else:
                    high_q_d.append(np.nanmean([i if i != 0 else np.NaN for i in [item for sublist in LEN_OF_SEQ[i] for item in sublist]]))
            else:
                high_q_d.append(np.NaN)

        print('High discharge duration calculation')

        return high_q_d
    
    
    ###########################################################################################################################################
    
    
    def low_q_frequency(Valid_gauges_in_mm): #Anthony Ladson version
    
        hydro_data = Valid_gauges_in_mm

        low_q_f = list()

        for gauge in hydro_data:
            if len(gauge) > 1:
                low_q_f.append(
                len(gauge.layer.dropna()[gauge.layer.dropna() < gauge.layer.dropna().mean() * 0.2])/len(gauge.layer.dropna()) * 365.25
                )
            else:
                low_q_f.append(np.NaN)

        print('High discharge frequency calculation')

        return low_q_f
    
    
    ###########################################################################################################################################
    
    
    def low_q_duration(Valid_gauges_in_mm, number_of_Nan):
    
        hydro_data = Valid_gauges_in_mm

        every_gauge_split_by_year_mm = split_by_year(hydro_data, number_of_Nan)

        FREQ_INSTANCE = list()
        SEQ_OF_FREQ = list()
        LEN_OF_SEQ = [[[] for year in gauge] for gauge in every_gauge_split_by_year_mm]


        for i, gauge in enumerate(every_gauge_split_by_year_mm):
            mean_x02 = hydro_data[i].layer.dropna().mean() * 0.2
            FREQ_INSTANCE.append([every_gauge_split_by_year_mm[i][j][every_gauge_split_by_year_mm[i][j].layer < mean_x02] 
                              for j in range(len(every_gauge_split_by_year_mm[i]))])

            SEQ_OF_FREQ.append([[d for _, d in FREQ_INSTANCE[i][j].groupby(FREQ_INSTANCE[i][j].index - np.arange(len(FREQ_INSTANCE[i][j])))]
                                if FREQ_INSTANCE[i][j].empty != True else [0]
                           for j in range(len(FREQ_INSTANCE[i]))])
            for k, seq in enumerate(SEQ_OF_FREQ[i]):
                if len(seq) == 0:
                    LEN_OF_SEQ[i][k] = [np.NaN]
                else:
                    LEN_OF_SEQ[i][k] = [len(seq[ii])
                                        if type(seq[ii]) != int
                                        else 0
                                        for ii in range(len(seq))]
        low_q_d = list()


        for i, gauge in enumerate(LEN_OF_SEQ):
            if len(gauge) > 1:
                if all([True if i == 0 else False for i in [item for sublist in LEN_OF_SEQ[i] for item in sublist]]):
                    low_q_d.append(0)
                else:
                    low_q_d.append(np.nanmean([i if i != 0 else np.NaN for i in [item for sublist in LEN_OF_SEQ[i] for item in sublist]]))
            else:
                low_q_d.append(np.NaN)

        print('Low discharge duration calculation')

        return low_q_d

    
    
    ###########################################################################################################################################
    
    
    def zero_q_frequency(Valid_gauges_in_mm):
        
        hydro_data = Valid_gauges_in_mm

        zero_q_freq = list()

        for gauge in hydro_data:
            if len(gauge) > 1:
                zero_q_freq.append(
                len(gauge.layer.dropna()[gauge.layer.dropna() == 0.0])/len(gauge.layer.dropna()) * 365.25
                )
            else:
                zero_q_freq.append(np.NaN)

        print('Zero discharge frequency')
        return zero_q_freq    
    
    ###########################################################################################################################################

    camels_ru_hydro = pd.DataFrame(data = [ID, 
                                           mean_for_gauge(list_of_gauges),
                                           slope_fdc_gauge(list_of_gauges),
                                           BFI_calculation_for_lists(list_of_gauges, 60)[0],
                                           Q5_for_gauge(list_of_gauges), 
                                           Q95_for_gauge(list_of_gauges),
                                           high_q_frequency(list_of_gauges), #my require 0
                                           high_q_duration(list_of_gauges, 0), 
                                           low_q_frequency(list_of_gauges), #my require 0
                                           low_q_duration(list_of_gauges, 0),
                                           zero_q_frequency(list_of_gauges),
                                           hfd_mean_for_gauges(list_of_gauges, 0)])
    camels_ru_hydro = camels_ru_hydro.T
    
    camels_ru_hydro.columns = ['gauge_id', 'q_mean', 'slope_fdc', 'baseflow_index', 'q5', 'q95',
                               'high_q_freq', 'high_q_dur', 'low_q_freq', 'low_q_dur', 'zero_q_freq', 'hfd_mean']
    camels_ru_hydro = camels_ru_hydro.applymap('{:.4f}'.format)
    camels_ru_hydro['gauge_id'] = ID
#     camels_ru_hydro.to_csv(path_to_results + 'camels_ru_hydro.csv', sep = ';', index = False)
    
    return camels_ru_hydro

In [7]:
hydro_signatures_for_CAMELS(hydro_data = hydrology_data, path_to_results = path_to_results + 'test', ID = ID)

Mean layer of discharge calculation
Flow duration curve calculation
Base flow index calculation
Расчёт на 0 посту закончилась за 13 секунд
Расчёт на 1 посту закончилась за 11 секунд
Расчёт на 2 посту закончилась за 9 секунд
Расчёт на 3 посту закончилась за 11 секунд
Расчёт на 4 посту закончилась за 11 секунд
Расчёт на 5 посту закончилась за 12 секунд
Расчёт на 6 посту закончилась за 9 секунд
Расчёт на 7 посту закончилась за 7 секунд
Расчёт на 8 посту закончилась за 11 секунд
Расчёт на 9 посту закончилась за 11 секунд
Расчёт на 10 посту закончилась за 6 секунд
Base Flow Index splitting
5% quantile calculation
95% quantile calculation
High discharge frequency calculation
High discharge duration calculation
High discharge frequency calculation
Low discharge duration calculation
Zero discharge frequency
Half flow date calculation


,gauge_id,q_mean,slope_fdc,baseflow_index,q5,q95,high_q_freq,high_q_dur,low_q_freq,low_q_dur,zero_q_freq,hfd_mean
0,72552,0.5039,2.6141,0.4992,0.0684,1.7404,7.8856,4.9394,41.4106,16.5524,0.0000,180.0750
1,72559,1.0262,2.1093,0.6024,0.2383,2.9562,1.5654,3.1818,9.3487,9.2500,0.0000,177.2326
2,72584,0.3761,3.3685,0.6874,0.0058,0.9847,0.0000,0.0000,83.4214,37.8667,0.1209,217.6562
3,72585,1.2275,1.3142,0.7893,0.4539,2.4685,0.0000,0.0000,1.0741,12.2500,0.0000,204.9070
4,72588,0.8675,1.3347,0.6903,0.3128,2.1252,1.5111,6.1818,0.0222,1.0000,0.0000,196.6744
5,72592,0.7401,2.5051,0.5408,0.1232,2.4317,8.0229,8.3864,34.0703,25.2742,0.0000,189.9545
6,72603,0.5292,3.4358,0.4492,0.0503,2.0127,18.2449,7.4176,97.9550,29.6435,0.0000,188.4286
7,72605,0.6734,3.1410,0.4700,0.0899,2.5795,15.9856,5.9178,68.5611,14.0075,0.0000,191.9615
8,72610,1.6280,2.1163,0.5954,0.3636,4.9268,3.1421,5.2593,10.9652,6.7143,0.0000,184.5238
9,72617,1.1986,2.0741,0.6634,0.2841,3.1755,0.1137,2.5000,10.5013,10.4048,0.0000,188.0476


In [7]:
def get_HydroATLAS_for_WS(WS, WS_index, path_to_HydroATLAS):
    """                             
    WS - Your data with watershed GDF.geometry[:]
    WS_index - Number of the index of WS from GeoDataFrame geometry field
    path_to_HydroATLAS - Path to BasinATLAS_v10.gdb file
    path_to_results - Path where calculations will be saved
    """
    import fiona 
    import geopandas as gpd
    from shapely.geometry import Polygon, MultiPolygon

    pd.options.mode.chained_assignment = None
    
    def polygon_area(lats, lons, radius = 6378137):
        """
        Computes area of spherical polygon, assuming spherical Earth. 
        Returns result in ratio of the sphere's area if the radius is specified.
        Otherwise, in the units of provided radius.
        lats and lons are in degrees.
        """
        from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
        lats = np.deg2rad(lats)
        lons = np.deg2rad(lons)

        # Line integral based on Green's Theorem, assumes spherical Earth

        #close polygon
        if lats[0]!=lats[-1]:
            lats = append(lats, lats[0])
            lons = append(lons, lons[0])

        #colatitudes relative to (0,0)
        a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
        colat = 2*arctan2( sqrt(a), sqrt(1-a) )

        #azimuths relative to (0,0)
        az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

        # Calculate diffs
        # daz = diff(az) % (2*pi)
        daz = diff(az)
        daz = (daz + pi) % (2 * pi) - pi

        deltas=diff(colat)/2
        colat=colat[0:-1]+deltas

        # Perform integral
        integrands = (1-cos(colat)) * daz

        # Integrate 
        area = abs(sum(integrands))/(4*pi)

        area = min(area,1-area)
        if radius is not None: #return in units of radius
            return area * 4 * pi* radius**2 / 10**6
        else: #return in ratio of sphere total area
            return area / 10**6

    def select_big_from_MP(WS_geometry):
        if type(WS_geometry) == MultiPolygon:
            big_area = [polygon_area(lats = polygon.exterior.coords.xy[1], 
                                    lons = polygon.exterior.coords.xy[0]) 
                        for polygon in WS_geometry]
            import numpy as np
            WS_geometry = WS_geometry[np.argmax(big_area)]
        else:
            WS_geometry = WS_geometry
        return WS_geometry
    
    # basic numbers for different variables
    monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    land_cover_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']
    natural_vegetation = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
    wetland_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09']
    
    # Get all the layers from the .gdb file 
    layers = fiona.listlayers(path_to_HydroATLAS)[-1]
    # -1 layer - high density sub-basins (lowest area)
    
    # Read choosen geodatabase layer with geopandas
    gdf = gpd.read_file(path_to_HydroATLAS, 
                        mask = WS.geometry[WS_index], layer=layers,  ignore_geometry=False)
    
    def filter_HydroATLAS_sub_basins(WS_own, HydroATLAS_data):

        """

        WS_own - Watershed from your GDF of watersheds
        HydroATLAS_data - gdf file from layers of geodatabase

        """

        # return biggest polygon from multipolygon array

        def select_big_from_MP(WS_geometry):
            if type(WS_geometry) == MultiPolygon:
                big_area = [polygon_area(lats = polygon.exterior.coords.xy[1], 
                                        lons = polygon.exterior.coords.xy[0]) 
                            for polygon in WS_geometry]
                import numpy as np
                WS_geometry = WS_geometry[np.argmax(big_area)]
            else:
                WS_geometry = WS_geometry
            return WS_geometry

        WS_own = select_big_from_MP(WS_own)
        ### WS from your data
        your_WS = gpd.GeoSeries([WS_own])

        ### Create extra gdf to use geopandas functions
        gdf_your_WS = gpd.GeoDataFrame({'geometry': your_WS})
        gdf_your_WS = gdf_your_WS.set_crs('EPSG:4326')

        intersected_sub_basins = list()



        for HydroATLAS_row in range(len(HydroATLAS_data)):

            # selection from sub-basins of GeoDataBase
            HydroATLAS_WS = gpd.GeoSeries([HydroATLAS_data.geometry[HydroATLAS_row]])        

            gdf_HydroATLAS_WS = gpd.GeoDataFrame({'geometry': HydroATLAS_WS}).set_crs('EPSG:4326')

            #intersect basins
            res_intersection = gpd.overlay(gdf_your_WS, gdf_HydroATLAS_WS, how='intersection')

            """
            Check if our intersection between sub-basin form HydroAtlas and our watershed is more than 0.6 of 
            sub-basin itself
            If not - than pass        
            """
            if len(res_intersection) != 0:
                # check if it is MultiPolygon
                if type(res_intersection.geometry[0]) == MultiPolygon:
                    if polygon_area(lats = res_intersection.geometry[0][0].exterior.coords.xy[1], 
                                    lons = res_intersection.geometry[0][0].exterior.coords.xy[0])/polygon_area(lats = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], 
                                                                                                    lons = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0]) > 0.1:


                        intersected_sub_basins.append(HydroATLAS_data.loc[HydroATLAS_row])

                    else:
                        pass
                # If it's not it is Polygon
                else:
                    if polygon_area(lats = res_intersection.geometry[0].exterior.coords.xy[1], 
                                    lons = res_intersection.geometry[0].exterior.coords.xy[0])/polygon_area(lats = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[1], 
                                                                                                    lons = gdf_HydroATLAS_WS.geometry[0][0].exterior.coords.xy[0]) > 0.1:


                        intersected_sub_basins.append(HydroATLAS_data.loc[HydroATLAS_row])

                    else:
                        pass
            else:
                pass

        return intersected_sub_basins
    
    list_of_goodies = filter_HydroATLAS_sub_basins(WS.geometry[WS_index], gdf)
  
    if len(list_of_goodies) != 0:
        list_of_goodies = gpd.GeoDataFrame(pd.DataFrame(list_of_goodies)).set_crs('EPSG:4326').reset_index(drop = True)
        from shapely.ops import unary_union
        union_geometry = gpd.GeoSeries(unary_union([i for i in list_of_goodies.geometry])).set_crs('epsg:4326')
        union_geometry = select_big_from_MP(union_geometry.geometry[0])

        """
        group columns by category (difference is the way of mathematical aggregation)

        e.g. classes will be aggrgated by mode value in sub-basins of the watershed

        other values will be calculated as a mean for selected watershed

        """
        # values which will be aggregated by mean
        columns_MEAN = [['inu_pc_ult'], ['lka_pc_use'], ['lkv_mc_usu'], ['rev_mc_usu'], ['dor_pc_pva'], ['gwt_cm_sav'], ['ele_mt_sav'], ['slp_dg_sav'],
                ['sgr_dk_sav'], ['tmp_dc_s{}'.format(i) for i in monthes], ['pre_mm_s{}'.format(i) for i in monthes], 
                ['pet_mm_s{}'.format(i) for i in monthes], ['aet_mm_s{}'.format(i) for i in monthes], ['snw_pc_s{}'.format(i) for i in monthes], 
                ['glc_pc_s{}'.format(i) for i in land_cover_classes], ['pnv_pc_s{}'.format(i) for i in natural_vegetation], ['wet_pc_s{}'.format(i) for i in wetland_classes], 
                ['for_pc_sse'], ['crp_pc_sse'], ['pst_pc_sse'], ['ire_pc_sse'], ['gla_pc_sse'], ['prm_pc_sse'], ['cly_pc_sav'], ['slt_pc_sav'], 
                ['snd_pc_sav'], ['soc_th_sav'], ['swc_pc_syr'], ['swc_pc_s{}'.format(i) for i in monthes], ['kar_pc_sse'], ['ero_kh_sav'], ['urb_pc_sse']]

        # values which will be aggregated by mode
        columns_MODE = [['clz_cl_smj'], ['cls_cl_smj'], ['glc_cl_smj'], ['pnv_cl_smj'],
                        ['wet_cl_smj'], ['tbi_cl_smj'], ['tec_cl_smj'], ['lit_cl_smj']]

        # values which will be aggregated by mean but need extra calculations
        # e.g. ari_ix need to be divided by 10, cmi by 100 etc.
        # for some reason values exceed treshold of the range
        columns_EXTRA_CALC_MEAN = [['ari_ix_sav'], ['cmi_ix_s{}'.format(i) for i in monthes], ['hft_ix_s93'], ['hft_ix_s09']]

        # split list of lists to needed columns
        columns_EXTRA_CALC_MEAN = [item for sublist in columns_EXTRA_CALC_MEAN for item in sublist]
        columns_MEAN = [item for sublist in columns_MEAN for item in sublist]
        columns_MODE = [item for sublist in columns_MODE for item in sublist]
        
        # dataframe for indexes

        df_EXTRA_CALC_MEAN = list_of_goodies[columns_EXTRA_CALC_MEAN]
        df_EXTRA_CALC_MEAN.loc[:, ['ari_ix_sav']] /= 10 # aridity index is the value between 0 and 100. In current version of HydroATLAS (v 1.0) it's vary between 0 and 1000
        
        df_EXTRA_CALC_MEAN.loc[:, ['cmi_ix_s{}'.format(i) for i in monthes]] /= 100 # aridity index is the value between -1 and 1. In current version of HydroATLAS (v 1.0) it's vary between -100 and 100

        df_EXTRA_CALC_MEAN = df_EXTRA_CALC_MEAN.mean()
        
        # dataframe for area values

        df_MEAN = list_of_goodies[columns_MEAN]
        df_MEAN.loc[:, ['tmp_dc_s{}'.format(i) for i in monthes]] /= 10 # in some regions on North-West Russia average value for Jan -83. I assume it's need to be divide by 10
        df_MEAN = df_MEAN.mean()
        

        #dataframe for classes

        df_MODE = list_of_goodies[columns_MODE]
        df_MODE = df_MODE.replace(-9999, np.NaN) # Это вопрос: может стоит оставить "отсутствующий класс" "мокрых земель"
        df_MODE = df_MODE.mode()
        
        
        list_of_frames = [df_EXTRA_CALC_MEAN, df_MEAN, df_MODE]
        
        for i in range(len(list_of_frames)):
            if type(list_of_frames[i]) == pd.Series:
                list_of_frames[i] = list_of_frames[i].to_frame().T
            else:
                pass
            
        def split_by_categories(df_ecm, df_me, df_mo):
    
            # basic numbers for different variables
            monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
            land_cover_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']
            natural_vegetation = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
            wetland_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09']

            hydrology_variables = [item for sublist in [['inu_pc_ult'], ['lka_pc_use'], ['lkv_mc_usu'],
                                                    ['rev_mc_usu'], ['dor_pc_pva'], ['gwt_cm_sav']]
                            for item in sublist]

            physiography_variables = [item for sublist in [['ele_mt_sav'], ['slp_dg_sav'], ['sgr_dk_sav']] 
                                    for item in sublist]

            climate_variables = [item for sublist in [['clz_cl_smj'], ['cls_cl_smj'], ['tmp_dc_s{}'.format(i) for i in monthes], 
                                                    ['pre_mm_s{}'.format(i) for i in monthes], ['pet_mm_s{}'.format(i) for i in monthes],
                                                    ['aet_mm_s{}'.format(i) for i in monthes], ['ari_ix_sav'],
                                                    ['cmi_ix_s{}'.format(i) for i in monthes], ['snw_pc_s{}'.format(i) for i in monthes]] 
                                for item in sublist]

            landcover_variables = [item for sublist in [['glc_cl_smj'], ['glc_pc_s{}'.format(i) for i in land_cover_classes], 
                                                        ['pnv_cl_smj'], ['wet_cl_smj'], ['wet_pc_s{}'.format(i) for i in wetland_classes],
                                                        ['for_pc_sse'], ['crp_pc_sse'], ['pst_pc_sse'], 
                                                        ['ire_pc_sse'], ['gla_pc_sse'], ['prm_pc_sse'], 
                                                        ['tbi_cl_smj'], ['tec_cl_smj']]
                                for item in sublist]

            soil_and_geo_variables = [item for sublist in [['cly_pc_sav'], ['slt_pc_sav'], ['snd_pc_sav'], 
                                                        ['soc_th_sav'], ['swc_pc_syr'], ['swc_pc_s{}'.format(i) for i in monthes],
                                                        ['lit_cl_smj'], ['kar_pc_sse'], ['ero_kh_sav']]
                                    for item in sublist]

            urban_variables = [item for sublist in [['urb_pc_sse'], ['hft_ix_s93'], ['hft_ix_s09']] for item in sublist]

            # dataframe of hydrology variables
            df_HYDRO = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in hydrology_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in hydrology_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in hydrology_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)
            # dataframe of physiography variables
            df_PHYSIO = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in physiography_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in physiography_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in physiography_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)

            # dataframe of climate variables
            df_CLIMATE = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in climate_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in climate_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in climate_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)
            # dataframe of physiography variables                       
            df_LANDCOVER = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in landcover_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in landcover_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in landcover_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)
            # dataframe of soil and geology variables
            df_SOIL_GEO = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in soil_and_geo_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in soil_and_geo_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in soil_and_geo_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)
            # dataframe of urban variables
            df_URBAN = pd.concat([
                                    df_ecm[
                                    df_ecm.columns[
                                    [True if i in urban_variables else False for i in df_ecm.columns]
                                                            ]],
                                    df_me[
                                    df_me.columns[
                                    [True if i in urban_variables else False for i in df_me.columns]
                                                            ]],
                                    df_mo[
                                    df_mo.columns[
                                    [True if i in urban_variables else False for i in df_mo.columns]
                                                            ]]
                                    ], axis = 1)
            return [df_HYDRO, df_PHYSIO, df_CLIMATE, df_LANDCOVER, df_SOIL_GEO, df_URBAN]
        
        fin = split_by_categories(df_ecm = list_of_frames[0], df_me = list_of_frames[1], df_mo = list_of_frames[2])
    
    else:
        list_of_goodies = np.NaN
        union_geometry = np.NaN
        fin = np.NaN 
    
        
    return fin, union_geometry

In [8]:
geometry_data

,gauge_name,ID,AREA,geometry,Area
0,Hrevica - Ivanovskoe,72617.0,311,"POLYGON ((29.12397 59.54101, 29.12505 59.54099...",310.172137
1,Izhora - Annolovo,72729.0,875,"POLYGON ((30.20356 59.71164, 30.20897 59.71151...",872.109881
2,Kovashi - Lendovshina,72552.0,722,"POLYGON ((29.41333 59.92602, 29.41551 59.92598...",719.851796
3,Lemovzha - Hotnezha,72605.0,954,"POLYGON ((29.41373 59.57100, 29.41696 59.57094...",951.271639
4,Oredezh - Bolshoe Zarechie,72584.0,282,"POLYGON ((29.58405 59.55626, 29.58620 59.55621...",280.782670
5,Oredezh - Chikino,72585.0,325,"POLYGON ((29.58409 59.55680, 29.58516 59.55678...",323.762296
6,Oredezh - Virica,72588.0,828,"POLYGON ((29.58405 59.55626, 29.58620 59.55621...",825.279453
7,Orlinka - Orlinka,72592.0,211,"POLYGON ((30.13084 59.33856, 30.13726 59.33841...",210.345804
8,Sista - Srednee Raykovo,72559.0,589,"POLYGON ((28.87325 59.75869, 28.88083 59.75858...",587.450235
9,Strelka - Oliki,NaN,39,"POLYGON ((29.94785 59.77180, 29.95109 59.77173...",39.230482


In [14]:
def calculate_HydroATLAS_multiple_WS(WS_geometry, path_to_HydroATLAS, path_to_results):
    """
    Purpose of this function is to obtain characteristics for multiple WS
    geometry of which are stored in GeoDataFrame with next rows:
    ID, geometry
    
    WS_geometry - GeoDataFrame with geometry
    path_to_HydroATLAS - path to .gdb file
    path_to_results - folder where results will be saved
    
    """
    
    my_headache = list()
    
    for row in range(len(WS_geometry)):
        my_headache.append(get_HydroATLAS_for_WS(WS = WS_geometry,
                            WS_index = row, 
                            path_to_HydroATLAS = path_to_HydroATLAS + 'BasinATLAS_v10.gdb'))
        print('geographic characteristics for {} calculated!'.format(WS_geometry.ID[row]))
    
    bool_array = list()
    # check if value exist. To not mess with ID assignments
    for i in range(len(my_headache)):
        if type(my_headache[i][0]) == float:
            bool_array.append(False)
        else:
            bool_array.append(True)
            
    # common procedure: for each WS ID - certain geographic characteristics
    VALID_ID = [ID for i, ID in enumerate(WS_geometry.ID) if bool_array[i]]

    VALID_HYDRO = [hydro[0][0] for i, hydro in enumerate(my_headache) if bool_array[i]]
    hydro_df = pd.concat(VALID_HYDRO).dropna().reset_index(drop = True)
    hydro_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    hydro_df.to_csv(path_to_results + 'hydro.csv', index = False)

    VALID_PHYSIO = [physio[0][1] for i, physio in enumerate(my_headache) if bool_array[i]]
    physio_df = pd.concat(VALID_PHYSIO).dropna().reset_index(drop = True)
    physio_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    physio_df.to_csv(path_to_results + 'physio.csv', index = False)

    VALID_CLIMATE = [climate[0][2] for i, climate in enumerate(my_headache) if bool_array[i]]
    climate_df = pd.concat(VALID_CLIMATE).dropna().reset_index(drop = True)
    climate_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    climate_df.to_csv(path_to_results + 'climate.csv', index = False)

    VALID_URBAN = [urban[0][5] for i, urban in enumerate(my_headache) if bool_array[i]]
    urban_df = pd.concat(VALID_URBAN).dropna().reset_index(drop = True)
    urban_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    urban_df.to_csv(path_to_results + 'urban.csv', index = False)

    VALID_LANDCOVER = [landcover[0][3] for i, landcover in enumerate(my_headache) if bool_array[i]]
    landcover_df = pd.concat(VALID_LANDCOVER).dropna(thresh = 4).reset_index(drop = True)
    landcover_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    landcover_df.to_csv(path_to_results + 'landcover.csv', index = False)

    VALID_SOIL_GEO = [soil_geo[0][4] for i, soil_geo in enumerate(my_headache) if bool_array[i]]
    soil_geo_df = pd.concat(VALID_SOIL_GEO).dropna().reset_index(drop = True)
    soil_geo_df.insert(loc = 0, column = 'ID', value = VALID_ID)
    soil_geo_df.to_csv(path_to_results + 'soil_geo.csv', index = False)

    VALID_GEOM_HydroATLAS = [geometry[1] for i, geometry in enumerate(my_headache) if bool_array[i]]
    geometry_df = gpd.GeoDataFrame(VALID_GEOM_HydroATLAS,
                                   columns={'geometry'}).reset_index(drop = True)
    geometry_df.insert(loc = 0, column = 'ID', value = VALID_ID)

    geometry_df.to_csv(path_to_results + 'geometry_HydroATLAS_subB.csv', index = False)
    
    return 'Check results folder !'

In [ ]:
calculate_HydroATLAS_multiple_WS(WS_geometry=geometry_data,
                                path_to_HydroATLAS=HydroATLAS_data_path,
                                path_to_results=path_to_results)

geographic characteristics for 72617.0 calculated!
geographic characteristics for 72729.0 calculated!


### Abbreviation explanation (before the slash is the category where this variable are related
##### inu_pc - Hydrology / Inundation Extent
##### lka_pc - Hydrology / Limnicity (% per area)
##### lkv_mc - Hydrology / Lake volume
##### rev_mc - Hydrology / Reservoir volume
##### dor_pc - Hydrology / Degree of Regulation
##### gwt_cm - Hydrology / Groundwater table depth
##### ele_mt - Physiography / Elevation
##### slp_dg - Physiography / Terrain slope
##### sgr_dk - Physiography / Stream gradient
##### clz_cl - Climate / Climate zones
##### cls_cl - Climate / Climate strata
##### tmp_dc - Climate / Air Temperature
##### pre_mm - Climate / Precipitation
##### pet_mm - Climate / Potential evapotraspiration
##### aet_mm - Climate / Actual evapotranspiration
##### ari_ix - Climate / Global aridity index
##### cmi_ix - Climate / Climate moisture index
##### snw_pc - Climate / Snow cover extent
##### glc_cl - Landcover / Land cover classes
##### glc_pc - Landcover / Land cover extent
##### pnv_cl - Landcover / Potential natural vegetation classes
##### wet_cl - Landcover / Wetland classes
##### wet_pc - Landcover / Wetland extent
##### for_pc - Landcover / Forest cover extent
##### crp_pc - Landcover / Cropland extent
##### pst_pc - Landcover / Pasture extent
##### ire_pc - Landcover / Irrigated area extent
##### gla_pc - Landcover / Glacier extent
##### prm_pc - Landcover / Permafrost extent
##### tbi_cl - Landcover / Terrestrial biomes
##### tec_cl - Landcover / Terresterial ecoregions 
##### cly_pc - Soils & Geology / Clay fraction in soil
##### slt_pc - Soils & Geology / Silt fraction in soil
##### snd_pc - Soils & Geology / Sand fraction in soil
##### soc_th - Soils & Geology / Organic carbon content in soil
##### swc_pc - Soils & Geology / Soil water content
##### lit_cl - Soils & Geology / Lithological classes
##### kar_pc - Soils & Geology / Karst area extent
##### ero_kh - Soils & Geology / Soil erosion
##### urb_pc - Anthropogenic / Urban extent
##### hft_ix_93 - Anthropogenic / Human footprint (year 1993)
##### hft_ix_09 - Anthropogenic / Human footprint (year 2009)

### Dimension explanation
##### pc- percentage of something
##### mc - million cubic meters
##### cm - centimeters
##### mt - meters or meters above sea level (m.a.s.l)
##### dk - decimeter per kilometer
##### cl - classes
##### dg - degrees
##### dc - degreec celsius °
##### mm - millimeters
##### ix - index value
##### kh - kilogram per hectare (kg/ha) per year
##### th - tonnes per hectare

### Prefix explanation
##### s - in sub-basin
##### p - at sub-basin pour point or at reach pour point
##### u - in total watershed
##### c - in reach catchment (i.e. the local catchment that drains directly into the reach)

### Statistical aggregation index
##### mj - spatial majority
##### se - spatial extent(%)
##### yr - annual average
##### s{01..12} - montly average
##### lt - long-term maximum
##### su - sum
##### va - value
##### av - average

#### Transition to name from number in _cl variables

In [18]:
xlsx = pd.ExcelFile('/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/HydroAtlas/Data/HydroATLAS_v10_Legends.xlsx')
my_cl_names = [i for i in xlsx.sheet_names if i in ['clz_cl', 'cls_cl', 'glc_cl', 'pnv_cl', 'wet_cl', 'tbi_cl', 'tec_cl', 'lit_cl']]
name_schemes = [pd.read_excel('/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/HydroAtlas/Data/HydroATLAS_v10_Legends.xlsx', 
                              sheet_name = i) for i in my_cl_names]

In [19]:
name_schemes[0]

,GEnZ_ID,GEnZ_Name,GEnZ_Code
0,1,Arctic 1,A
1,2,Arctic 2,B
2,3,Extremely cold and wet 1,C
3,4,Extremely cold and wet 2,D
4,5,Cold and wet,E
5,6,Extremely cold and mesic,F
6,7,Cold and mesic,G
7,8,Cool temperate and dry,H
8,9,Cool temperate and xeric,I
9,10,Cool temperate and moist,J


### Read Open Forecast shape data

In [5]:
big_shape = gpd.read_file(
    '/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/openf_gauges_watersheds/watersheds_openf.shp'
)
big_shape = big_shape[['code', 'name_en', 'geometry']]
big_shape = big_shape.rename(columns={"code": "ID"})
big_shape


    [ - LIST OF HydroATLAS vars Which are List of DF for Each characteristic
         [
             - different characteristics in strict order
         ]
     ] - len of this equal to big_shape.ID as well as order

    EZ SAVE


### Проверка осреднений

In [57]:
df_MEAN.loc[:, ['tmp_dc_s{}'.format(i) for i in monthes]] # tmp / 10 ✓

,tmp_dc_s01,tmp_dc_s02,tmp_dc_s03,tmp_dc_s04,tmp_dc_s05,tmp_dc_s06,tmp_dc_s07,tmp_dc_s08,tmp_dc_s09,tmp_dc_s10,tmp_dc_s11,tmp_dc_s12
0,-80.666667,-76.666667,-33.666667,36.666667,103.666667,152.666667,170.0,154.0,103.666667,48.0,-8.666667,-52.666667


In [62]:
df_MEAN.loc[:, ['pre_mm_s{}'.format(i) for i in monthes]]

,pre_mm_s01,pre_mm_s02,pre_mm_s03,pre_mm_s04,pre_mm_s05,pre_mm_s06,pre_mm_s07,pre_mm_s08,pre_mm_s09,pre_mm_s10,pre_mm_s11,pre_mm_s12
0,40.0,30.666667,36.666667,38.666667,44.666667,70.333333,79.333333,86.666667,77.333333,69.0,65.333333,55.666667


In [63]:
df_MEAN.loc[:, ['sgr_dk_sav']]

,sgr_dk_sav
0,27.666667


In [66]:
df_MEAN.loc[:, ['snw_pc_s{}'.format(i) for i in monthes]]

,snw_pc_s01,snw_pc_s02,snw_pc_s03,snw_pc_s04,snw_pc_s05,snw_pc_s06,snw_pc_s07,snw_pc_s08,snw_pc_s09,snw_pc_s10,snw_pc_s11,snw_pc_s12
0,94.666667,99.333333,89.0,20.333333,1.333333,1.666667,0.0,1.0,3.333333,24.666667,64.333333,91.666667


In [67]:
df_MEAN.loc[:, ['kar_pc_sse']]

,kar_pc_sse
0,0.0


### TEST ORIGINAL CAMELS


In [16]:
# Расчёт произведён только для одного водосбора. Если на нём всё получилось - получиться и на всех

camels_test_gauge = pd.read_csv('/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/basin_dataset_public_v1p2/usgs_streamflow/01/01013500_streamflow_qc.txt', 
                               names = ['ID', 'year', 'month', 'day', 'discharge', 'idk'], delim_whitespace=True)
camels_test_gauge['date'] = pd.to_datetime(camels_test_gauge[['year', 'month', 'day']])
camels_test_gauge = camels_test_gauge.drop(['year', 'month', 'day'], axis = 1)
camels_test_gauge['discharge'] = camels_test_gauge['discharge'].replace(-999, np.NaN)
camels_test_gauge['discharge'] *= 0.0283168

camels_test_gauge = camels_test_gauge.set_index('date', drop = False)['1989-10-01' :'2009-09-30']

In [17]:
camels_AREA_for_test_gauge = pd.read_csv('/mnt/c/education/HSI/aspirantura/CAMELS_ru/files/basin_dataset_public_v1p2/basin_metadata/gauge_information.txt', 
                                         sep = "\t+", 
                                         nrows = 2, 
                                         engine = 'python').reset_index(drop = True)
camels_AREA_for_test_gauge

,HUC_02 GAGE_ID,GAGE_NAME,LAT,LONG,DRAINAGE AREA (KM^2)
0,1013500,"Fish River near Fort Kent, M...",47.23739,-68.58264,2252.7
1,1022500,"Narraguagus River at Cherryfield, M...",44.60797,-67.93524,573.6


In [18]:
camels_test_gauge['layer'] = (86400 * camels_test_gauge['discharge'] * 10**9) / (camels_AREA_for_test_gauge['DRAINAGE AREA (KM^2)'][0] * 10**12)
camels_test_gauge

,ID,discharge,idk,date,layer
date,,,,,
1989-10-01,1013500,8.495040,A,1989-10-01,0.325819
1989-10-02,1013500,8.410090,A,1989-10-02,0.322560
1989-10-03,1013500,8.863158,A,1989-10-03,0.339937
1989-10-04,1013500,8.381773,A,1989-10-04,0.321474
1989-10-05,1013500,8.098605,A,1989-10-05,0.310614
...,...,...,...,...,...
2009-09-26,1013500,8.041971,A,2009-09-26,0.308442
2009-09-27,1013500,8.013654,A,2009-09-27,0.307356
2009-09-28,1013500,8.749891,A,2009-09-28,0.335593


In [218]:
hydro_signatures_for_CAMELS(hydro_data = [camels_test_gauge], path_to_results = path_to_results + 'test',
                            ID = [camels_test_gauge['ID'][1]])

Mean layer of discharge calculation
Flow duration curve calculation
Base flow index calculation
Расчёт на 0 посту закончилась за 7 секунд
Base Flow Index splitting
5% quantile calculation
95% quantile calculation
High discharge frequency calculation
High discharge duration calculation
High discharge frequency calculation
Low discharge duration calculation
Zero discharge frequency
Half flow date calculation


,gauge_id,q_mean,slope_fdc,baseflow_index,q5,q95,high_q_freq,high_q_dur,low_q_freq,low_q_dur,zero_q_freq,hfd_mean
0,1013500,1.6992,2.6965,0.5357,0.2411,6.3730,6.1000,8.7143,41.3500,20.1707,0.0000,207.2500


In [15]:
pd.read_csv('/mnt/c/education/HSI/aspirantura/CAMELS_ru/literature/camels_attributes_v2.0/camels_hydro.txt', nrows = 2, sep = ';').iloc[0].to_frame().T

,gauge_id,q_mean,runoff_ratio,slope_fdc,baseflow_index,stream_elas,q5,q95,high_q_freq,high_q_dur,low_q_freq,low_q_dur,zero_q_freq,hfd_mean
0,1013500.0,1.699155,0.543437,1.528219,0.585226,1.845324,0.241106,6.373021,6.1,8.714286,41.35,20.170732,0.0,207.25


In [16]:
import copy
import math

In [17]:
test = copy.deepcopy(camels_test_gauge.layer)
test = test.reset_index(drop = True).to_frame()
test = test.sort_values('layer', ascending = False)
test['m'] = [i+1 for i in range(len(test))]
test['P'] = [100 * (i/7306) for i in test['m']]
test = test.reset_index(drop = True)

In [18]:
test[test['P'] > 33].reset_index(drop = True)

,layer,m,P
0,1.520487,2411,33.000274
1,1.520487,2412,33.013961
2,1.520487,2413,33.027649
3,1.520487,2414,33.041336
4,1.520487,2415,33.055023
...,...,...,...
4890,0.046701,7301,99.931563
4891,0.045615,7302,99.945250
4892,0.045615,7303,99.958938
4893,0.045615,7304,99.972625


In [19]:
q_33 = test[test['P'] > 33].reset_index(drop = True).loc[0, 'layer']

In [28]:
q_33, np.nanpercentile(camels_test_gauge.layer.dropna().to_numpy(), q = 100 - 33)

(1.520486584099081, 1.520486584099081)

In [21]:
q_66 = test[test['P'] > 66].reset_index(drop = True).loc[0, 'layer']

In [29]:
q_66, np.nanpercentile(camels_test_gauge.layer.dropna().to_numpy(), q = 100 - 66)

(0.6244855613264083, 0.6244855613264083)

In [30]:
(math.log(np.nanpercentile(camels_test_gauge.layer.dropna().to_numpy(), q = 100 - 33)) - 
 math.log(np.nanpercentile(camels_test_gauge.layer.dropna().to_numpy(), q = 100 - 66))) / (0.66 - 0.33)

2.6965378024424225

In [346]:
def high_q_frequency_old(Valid_gauges_in_mm, number_of_Nan): # MY
        
    hydro_data = Valid_gauges_in_mm
    
    every_gauge_split_by_year_mm = split_by_year(hydro_data, number_of_Nan)

    temp_high_q_f = [[] for _ in every_gauge_split_by_year_mm]

    for i, gauge in enumerate(every_gauge_split_by_year_mm):
        median_x9 = hydro_data[i].layer.dropna().median() * 9
        for year in gauge:
            if len(gauge) > 1:
                temp_high_q_f[i].append(len(year[year.layer > median_x9])/len(year)*100)
            else:
                temp_high_q_f[i].append(np.NaN)

    high_q_f = list()
    for gauge in temp_high_q_f:
        if len(gauge) > 1:
            high_q_f.append(np.nanmean([np.NaN if i == 0 else i for i in gauge]))
        else:
            high_q_f.append(np.NaN)

    print('High discharge frequency calculation Abramov')
    
    return high_q_f

def low_q_frequency_old(Valid_gauges_in_mm, number_of_Nan): # MY
        
    hydro_data = Valid_gauges_in_mm
    
    every_gauge_split_by_year_mm = split_by_year(hydro_data, number_of_Nan)

    temp_high_q_f = [[] for _ in every_gauge_split_by_year_mm]

    for i, gauge in enumerate(every_gauge_split_by_year_mm):
        mean_x02 = hydro_data[i].layer.dropna().mean() * 0.2
        for year in gauge:
            if len(gauge) > 1:
                temp_high_q_f[i].append(len(year[year.layer < mean_x02])/len(year)*100)
            else:
                temp_high_q_f[i].append(np.NaN)

    low_q_f = list()
    for gauge in temp_high_q_f:
        if len(gauge) > 1:
            low_q_f.append(np.nanmean([np.NaN if i == 0 else i for i in gauge]))
        else:
            low_q_f.append(np.NaN)

    print('Low discharge frequency calculation Abramov')
    
    return low_q_f

In [347]:
def high_q_frequency(Valid_gauges_in_mm): #Anthony Ladson version
    
    hydro_data = Valid_gauges_in_mm
    
    high_q_f = list()
    
    for gauge in hydro_data:
        if len(gauge) > 1:
            high_q_f.append(
            len(gauge[gauge.layer.dropna() > gauge.layer.dropna().median() * 9])/len(gauge.layer.dropna()) * 365.25
            )
        else:
            high_q_f.append(np.NaN)

    print('High discharge frequency calculation Ladson')

    return high_q_f

def low_q_frequency(Valid_gauges_in_mm): #Anthony Ladson version
    
    hydro_data = Valid_gauges_in_mm
    
    low_q_f = list()
    
    for gauge in hydro_data:
        if len(gauge) > 1:
            low_q_f.append(
            len(gauge[gauge.layer.dropna() < gauge.layer.dropna().mean() * 0.2])/len(gauge.layer.dropna()) * 365.25
            )
        else:
            low_q_f.append(np.NaN)

    print('High discharge frequency calculation Ladson')

    return low_q_f

In [348]:
high_q_frequency_old([camels_test_gauge], 0)

High discharge frequency calculation Abramov


[2.844444773481378]

In [349]:
high_q_frequency([camels_test_gauge])

High discharge frequency calculation Ladson


[6.1]

In [350]:
low_q_frequency_old([camels_test_gauge], 0)

Low discharge frequency calculation Abramov


[12.979052556333396]

In [351]:
low_q_frequency([camels_test_gauge])

High discharge frequency calculation Ladson


[41.35]